In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#!/usr/bin/env python3
import os
import json
import pandas as pd
from google import genai
from config import PROFILE_CSV_PATH, QUESTION_LIST_PATH
from engine.qa_generator import batch_questions, personalize_question
from prompts.prompt_builder import build_prompt_and_parser

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

usage_log = []
_original = client.models.generate_content

def _patched_generate_content(*args, **kwargs):
    resp = _original(*args, **kwargs)
    u = resp.usage_metadata
    usage_log.append({
        "prompt_tokens":     u.prompt_token_count,
        "completion_tokens": u.candidates_token_count,
        "total_tokens":      u.total_token_count,
    })
    return resp

client.models.generate_content = _patched_generate_content

# ─────────────────────────────────────────────────────────────────────────────
# 3) Load profiles & questions
# ─────────────────────────────────────────────────────────────────────────────
df_profiles = pd.read_csv('data/student_profiles.csv', encoding='utf-8')
questions   = [q.strip() for q in open('data/questions.txt', "r", encoding="utf-8") if q.strip()]
question_batches = batch_questions(questions, batch_size=10)

prompt, parser = build_prompt_and_parser()


def run_one_batch(profile_row: dict, batch: list[str]) -> dict:
    student_id = profile_row.get("student_id", "unknown_student")
    # 5a) Personalize each question
    personalized = [personalize_question(q, student_id) for q in batch]
    # 5b) Build the question block string
    question_block = "\n".join(f"{i+1}. {q}" for i, q in enumerate(personalized))
    # 5c) Build prompt text exactly
    # Unpack the dict:
    values = {
        "profile":   json.dumps(profile_row, indent=2),
        "questions": question_block
    }
    prompt_text = prompt.format(**values)

    # 5d) Call Gemini (logged by our patch)
    resp = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=prompt_text
    )
    raw = resp.text
    try:
        parsed = parser.parse(raw)
    except Exception:
        parsed = None  # we don't actually need the parsed output here
    return raw

usage_log.clear()
sample = df_profiles.iloc[0].to_dict()
run_one_batch(sample, question_batches[0])

assert len(usage_log) == 1, "Expected exactly one generate_content call"
pilot = usage_log[0]
n_qa_prompt = pilot["prompt_tokens"]
n_qa_out    = pilot["completion_tokens"]
n_qa_total  = pilot["total_tokens"]

print("Pilot Q&A token usage:")
print(f"  prompt_tokens     = {n_qa_prompt}")
print(f"  completion_tokens = {n_qa_out}")
print(f"  total_tokens      = {n_qa_total}")

N_profiles   = 1000            # e.g. 1000
batches_each = len(question_batches)       # e.g. 4
N_calls      = N_profiles * batches_each   # total batch calls

estimated_QA_tokens = N_calls * n_qa_total

print(f"\nTotal calls: {N_calls}")
print(f"Estimated Q&A token usage for {N_profiles} profiles, {batches_each} batches each: of 10 questions each")
print(f"Estimated total Q&A tokens: {estimated_QA_tokens:,}")
print(f"  → Prompt-only:    {N_calls * n_qa_prompt:,}")
print(f"  → Completion-only:{N_calls * n_qa_out:,}")


Pilot Q&A token usage:
  prompt_tokens     = 1230
  completion_tokens = 322
  total_tokens      = 1552

Total calls: 5000
Estimated Q&A token usage for 1000 profiles, 5 batches each:
Estimated total Q&A tokens: 7,760,000
  → Prompt-only:    6,150,000
  → Completion-only:1,610,000
